# PRODUCTS

**A. CLEAN UP AND PREPARE PRODUCT DATA**

A.1 Load and plot

In [ ]:
%matplotlib inline
import pandas as pd

import matplotlib.pyplot as plt

import os

# pip install fuzzywuzzy
from fuzzywuzzy import fuzz

from operator import itemgetter

import re

import seaborn as sns

#Website object lists : https://7esl.com/tools-and-equipment/#Technological_Gadgets

In [ ]:
os.getcwd()

In [ ]:
#load the data
hh_demographic = pd.read_csv('../data/dunnhumby_complete_csv/hh_demographic.csv', sep = ',')

transaction_data = pd.read_csv('../data/dunnhumby_complete_csv/transaction_data.csv', sep = ',')

product = pd.read_csv('../data/dunnhumby_complete_csv/product.csv', sep = ',')

# This is a list a created. It is a small file, I pushed it in data in order you can have it at home
ObjectLabels = pd.read_csv('../data/dunnhumby_complete_csv/GroceryLists.csv', sep = ',')
ObjectLabels.columns = ['ITEM','LABEL']

First we get interested by the different departments in the commercial center. We reorder the departments and plot the number of different products we can find per department. 

In [ ]:
# REORDERING PART
ordered_manufacturer= product.MANUFACTURER.astype(int).sort_values()
ordered_manufacturer= ordered_manufacturer.unique()

product['MANUFACTURER'] = pd.Categorical(product['MANUFACTURER'],
                      ordered = True,
                      categories = ordered_manufacturer)

print ('The order of the manufacturer categories is :', ordered_manufacturer)

# remove empty products
product= product.drop(product[product['DEPARTMENT']== ' '].index)

In [ ]:
# We drop weird department data. These data are either incomprehensible or appear to be inaccurate
product= product.drop(product[product['DEPARTMENT']== ' '].index)
product= product.drop(product[product['DEPARTMENT']== 'PHOTO'].index)
product= product.drop(product[product['DEPARTMENT']== 'VIDEO'].index)
product= product.drop(product[product['DEPARTMENT']== 'MISC. TRANS.'].index)
product= product.drop(product[product['DEPARTMENT']== 'MISC SALES TRAN'].index)
product= product.drop(product[product['DEPARTMENT']== 'MEAT-WHSE'].index)
# this department is the coupon store. Our study does not take into account the coupons
product= product.drop(product[product['DEPARTMENT']== 'COUP/STR & MFG'].index)

In [ ]:
#we create the LABEL column in the product dataframe
product['LABEL'] = product['DEPARTMENT']
product.head(4)

In [ ]:
department= product.groupby(['DEPARTMENT']).count()

#PLOT THE NUMBER OF DIFFERENT PRODUCTS PER DEPARTMENT

fig = plt.figure(figsize=(20,10))

# Divide the figure into a 2x1 grid, and give me the first section
ax1 = fig.add_subplot(231)

# Divide the figure into a 2x1 grid, and give me the second section
ax2 = fig.add_subplot(232)

ax3 = fig.add_subplot(233)

ax4 = fig.add_subplot(234)

ax5 = fig.add_subplot(235)

ax6 = fig.add_subplot(236)

department[(department.BRAND>0) & (department.BRAND<=2)].BRAND.sort_values().plot.bar(ax=ax1)
department[(department.BRAND>2) & (department.BRAND<=5)].BRAND.sort_values().plot.bar(ax=ax2)
department[(department.BRAND>5) & (department.BRAND<=20)].BRAND.sort_values().plot.bar(ax=ax3)
department[(department.BRAND>20) & (department.BRAND<=500)].BRAND.sort_values().plot.bar(ax=ax4)
department[(department.BRAND>500) & (department.BRAND<=4000)].BRAND.sort_values().plot.bar(ax=ax5)
department[department.BRAND>4000].BRAND.sort_values().plot.bar(ax=ax6)

fig.subplots_adjust(hspace=1)
fig.subplots_adjust(wspace=0.4)


As we can see in the previous barplots, the departments with the highest number of referenced products in this study are the grocery with almost 40000 products, and the drug GM (which is probably a drug store) with more than 30000 products. We are going to focus on these 2 mains stores in a first time.

**B. GROCERY DATA PREPROCESSING**

In this part we are interested to clean up and categorize the different products from the grocery store. We want to explain what purpose they serve (food, households, & hygien products, pet products etc.). We create a special Grocery dataframe for this purpose.

In [ ]:
#we extract the grocery product data which is the main supermarket in the commercial center
Grocery= pd.DataFrame(product[product['DEPARTMENT'] == 'GROCERY'])
Grocery.head()

The only product descriptions provided by the Grocery data are few words descriptions found in COMMODITY_DESC and SUB_COMMODITY_DESC columns. These descriptions are not standardized, no categories arise from them. 

In a first time, we want to categorize the words in the descriptions using different labels (produce, households, technology ...). To do so we create a function called "LabelizeProduct". This function first recognizes the different words or words sequences in the description. These words or words sequences are associated with a certain label in a list of shopping items ("GroceryList.csv"). In a second time, based on the fuzzywuzzy librairy, the "LabelizeProduct" gives a score with comparing the description and the shopping item corresponding to words sequence identified. Finally, the functions keep the label for which the score is the highest. 

In [ ]:
# This function is able to categorize the products given in data with looking at a labelled list of words. 
#This labelled list of words is a lexique of grocery and shopping words. The threshold is the smallest 
#fuzzywuzzy score accepted to declare a match. (fuzzywuzzy gives a score of string matching over 100 )

# data is the  dataset to categorize, dataCol is the column of the dataset to categorize, 
#threshold is the minimal fuzzywuzzy score accepted. If score lower than threshold, 
#the item is categorized as 'not found'. 
#wordList is the list of words associated with a Label. wordList : ['ITEM','LABEL']
def LabelizeProduct(data, dataCol, threshold, wordList) :
    
    Out = pd.DataFrame(index = data.index, columns = ['Item','Ratio','Label'])
    
    #count to show progression when running (can take few minutes)
    loopCounter = 0
    # iterate on the index of the dataset to categorize
    for indexCOM in data.index :
        
        if (loopCounter%100 == 0): 
            print(loopCounter)
        
        # the item to categorize in the dataCol
        item = dataCol.loc[indexCOM]
        #a vector to keep all the matches within the grocerylist.csv
        match = []
        #set the output table
        Out.Item[indexCOM] = item
        Out.Label[indexCOM] = 'not found'
        Out.Ratio[indexCOM]= 0
        
        #iterate on the wordList's items (belonging to a certain label)
        for index,label in enumerate(wordList.LABEL) :
            
            # get all the separated words in the wordList item
            line = re.split(' |&|/',wordList.ITEM[index])
            
            #iterate on the separated words in the wordList item
            for word in line:
                #manage case (everything in lowercase)
                lword= word.lower()
                litem = item.lower()

                # if we find the word from wordList item, in the description of the data:
                # we calculate the fuzzywuzzy ratio(%) and we append the match in the match vector
                if litem.find(lword)!= -1:
                    ratio = fuzz.token_sort_ratio(str.lower(item), str.lower(lword))
                    match.append((indexCOM,line,ratio,label))
                    
        # if the match vector is not empty, then we attribute the label corresponding to the max ratio 
        #to the description            
        if (match) and (max(match,key=itemgetter(2))[2]>= threshold)  :
            Out.Label[indexCOM] = max(match,key=itemgetter(2))[3]
            Out.Ratio[indexCOM]= max(match,key=itemgetter(2))[2]
            
        loopCounter += 1
            
    return Out

We use our new function "LabelizeProduct" to find the main categories in the Grocery department.

In [ ]:
LabelledSUB = LabelizeProduct(Grocery, Grocery.SUB_COMMODITY_DESC, 0, ObjectLabels)

In [ ]:
Labelled = LabelizeProduct(Grocery, Grocery.COMMODITY_DESC, 0, ObjectLabels) 

In [ ]:
Labelled['RatioSUB']= LabelledSUB.Ratio
Labelled['LabelSUB']= LabelledSUB.Label

In [ ]:
# we would like to estimate the error

# number of item where SUB_COMMODITY_DESC and COMMODITY_DESC have a different label. (Knowing the the label kept 
# at the end is the one with the highest score 
Labelled[Labelled['Label']!=Labelled['LabelSUB']].count()

# total of not found for both SUB_COMMODITY_DESC and COMMODITY_DESC
Labelled[(Labelled['Label'] == 'not found') & (Labelled['LabelSUB'] == 'not found')].count()

# Total number of highest score lower than 20. Most often it is because we compare a small word to 
# a whole item description (several words).  
Labelled[(Labelled['Ratio']<20) & (Labelled['RatioSUB'] < 20)].count()
Labelled[(Labelled['Ratio']<20) & (Labelled['RatioSUB'] < 20)]

We create a second function "putLabel" that sets the label with the highest fuzzy ratio in between the dataset's columns that were categorized (here COMMODITY_DESC and SUB_COMMODITY_DESC) in the main data set (here Grocery). 

In [ ]:
# this is the function that put the Labels in the main data Table. It compares the labels thanks to the fuzzy scores
#for 2 different labelledData (ex: grocery labelled for COMMODITY and SUB_COMMODITY) and chooses the best one. 
def putLabel(LabelledData, data) :
    for line in LabelledData.index : 

        maxRatio = max(LabelledData.Ratio.loc[line],LabelledData.RatioSUB.loc[line])
        if LabelledData.Ratio.loc[line] == maxRatio :
            data['LABEL'].loc[line] = LabelledData.Label.loc[line]
        elif LabelledData.RatioSUB.loc[line] == maxRatio :
            data['LABEL'].loc[line] = LabelledData.LabelSUB.loc[line]
    return 0

In [ ]:
# put the labels in the main data set Grocery and show results
putLabel(Labelled, Grocery)
LabelCount= Grocery.groupby('LABEL').count()
LabelCount.BRAND.sort_values().plot.bar()

In [ ]:
Grocery.to_csv('Grocery.csv')

As we can see in the following table the Grocery store product have been Labelled. Error of classification are possible (even if low rate --> to estimate, HOW ?). Indeed the "LabelizeProduct" recognizes words that are provided in the "wordList" to the function. The accuracy of the classification relies on the thoroughness of this word list. Moreover if the word list does not contain enough words, the quantity of 'not found' increases drastically.

In [ ]:
Grocery.groupby('LABEL').count()

**C. LET'S DO IT AGAIN WITH DRUG GM**

In [ ]:
#we extract the drug gm product data which is the second main store in the commercial center
DrugGM= pd.DataFrame(product[product['DEPARTMENT'] == 'DRUG GM'])
DrugGM.head()

In [ ]:
# use LabelizeProduct to label the COMMODITY_DESC column from Drug GM department
LabelledDGM = LabelizeProduct(DrugGM, DrugGM.COMMODITY_DESC, 0,ObjectLabels) 

In [ ]:
# use LabelizeProduct to label the SUB_COMMODITY_DESC column from Drug GM department
LabelledSUBDGM = LabelizeProduct(DrugGM, DrugGM.SUB_COMMODITY_DESC, 0,ObjectLabels) 

In [ ]:
# put the labels in the main data set Drug GM
LabelledDGM['RatioSUB']= LabelledSUBDGM.Ratio
LabelledDGM['LabelSUB']= LabelledSUBDGM.Label
putLabel(LabelledDGM, DrugGM)

In [ ]:
# show results
LabelDGMCount= DrugGM.groupby('LABEL').count()
LabelDGMCount.BRAND.sort_values().plot.bar()

In [ ]:
# we would like to estimate the error

# number of item where SUB_COMMODITY_DESC and COMMODITY_DESC have a different label. (Knowing the the label kept 
# at the end is the one with the highest score  
LabelledDGM[LabelledDGM['Label']!=LabelledDGM['LabelSUB']].count()

# total of not found for both SUB_COMMODITY_DESC and COMMODITY_DESC
LabelledDGM[(LabelledDGM['Label'] == 'not found') & (LabelledDGM['LabelSUB'] == 'not found')].count()

# Total number of highest score lower than 20. Most often it is because we compare a small word to 
# a whole item description (several words).  
LabelledDGM[(LabelledDGM['Ratio']<20) & (LabelledDGM['RatioSUB'] < 20)].count()
LabelledDGM[(LabelledDGM['Ratio']<60) & (LabelledDGM['RatioSUB'] < 20)]

**D. ALL THE OTHER SHOPS: DELI, PRODUCE, NUTRITION, COSMETICS etc.**

The shop DELI needs also to be categorized. Since all the other shops are specialized the labels are easy to infer. Indeed it corresponds to the shop specialization. We also put the computed Label in the main dataset product (Grocery, Drug GM and DELI). 

In [ ]:
# categorize the data in DELI
DELI = product[product['DEPARTMENT'] == 'DELI']
LabelledDELI = LabelizeProduct(DELI, DELI.COMMODITY_DESC, 0,ObjectLabels)
LabelledSUBDELI = LabelizeProduct(DELI, DELI.SUB_COMMODITY_DESC, 0,ObjectLabels)

In [ ]:
#add DELI labels in the product dataframe and show results
LabelledDELI['RatioSUB']= LabelledSUBDELI.Ratio
LabelledDELI['LabelSUB']= LabelledSUBDELI.Label
putLabel(LabelledDELI, DELI)

In [ ]:
DELILabelCount= DELI.groupby('LABEL').count()
DELILabelCount.BRAND.sort_values().plot.bar()

In [ ]:
# we standardize some labels for the seek of the vizualization. 
# I had trouble with the label setting in the main data set product. This is a shitty implementation but working. 
#I have to figure it out

product.DEPARTMENT.loc[product['DEPARTMENT']== 'DAIRY DELI'] = 'DELI'

product.LABEL.loc[product['DEPARTMENT']== 'MEAT'] = 'MEAT & SEAFOOD'
product.LABEL.loc[product['DEPARTMENT']== 'MEAT-PCKGD'] = 'MEAT & SEAFOOD'
product.LABEL.loc[product['DEPARTMENT']== 'SEAFOOD'] = 'MEAT & SEAFOOD'
product.LABEL.loc[product['DEPARTMENT']== 'SEAFOOD-PCKGD'] = 'MEAT & SEAFOOD'
product.LABEL.loc[product['DEPARTMENT']== 'FROZEN GROCERY'] = 'FROZEN'
product.LABEL.loc[product['DEPARTMENT']== 'PHARMACY SUPPLY'] = 'PHARMACY'
product.LABEL.loc[product['DEPARTMENT']== 'PORK'] = 'MEAT & SEAFOOD'
product.LABEL.loc[product['DEPARTMENT']== 'RX'] = 'PHARMACY'
product.LABEL.loc[product['DEPARTMENT']== 'HOUSEWARES'] = 'not found'
product.LABEL.loc[product['DEPARTMENT']== 'ELECT &PLUMBING'] = 'not found'
product.LABEL.loc[product['DEPARTMENT']== 'HBC'] = 'not found'
product.LABEL.loc[product['DEPARTMENT']== 'DELI/SNACK BAR'] = 'not found'
product.LABEL.loc[product['DEPARTMENT']== 'CHARITABLE CONT'] = 'not found'
product.LABEL.loc[product['DEPARTMENT']== 'PROD-WHS SALES'] = 'not found'
product.LABEL.loc[product['DEPARTMENT']== 'GM MERCH EXP'] = 'not found'
product.LABEL.loc[product['DEPARTMENT']== 'CNTRL/STORE SUP'] = 'not found'
product.LABEL.loc[product['DEPARTMENT']== 'GRO BAKERY'] = 'BAKERY'
product.LABEL.loc[product['DEPARTMENT']== 'PASTRY'] = 'BAKERY'
product.LABEL.loc[product['DEPARTMENT']== 'AUTOMOTIVE'] = 'STATIONERY'
product.LABEL.loc[product['DEPARTMENT']== 'POSTAL CENTER'] = 'FAST FOOD & SNACKS'
product.LABEL.loc[product['DEPARTMENT']== 'CHEF SHOPPE'] = 'FAST FOOD & SNACKS'
product.LABEL.loc[product['DEPARTMENT']== 'TRAVEL & LEISUR'] = 'PRODUCE'
product.LABEL.loc[product['DEPARTMENT']== 'SALAD BAR'] = 'RESTAURANT'
product.LABEL.loc[product['DEPARTMENT']== 'GARDEN CENTER'] = 'FLORAL'
product.LABEL.loc[product['DEPARTMENT']== 'KIOSK-GAS'] = 'GASOLINE'

product.LABEL.loc[product['DEPARTMENT'] == 'GROCERY'] = Grocery.LABEL
product.LABEL.loc[product['DEPARTMENT'] == 'DRUG GM'] = DrugGM.LABEL
product.LABEL.loc[product['DEPARTMENT'] == 'DELI'] = DELI.LABEL

product.LABEL.unique()

We produce a final plot to show our results.

In [ ]:
ProductLabels= product.groupby('LABEL').count()
ProductLabels2= product.groupby(by= ['DEPARTMENT','LABEL']).count()
ProductDepartment= product.groupby(by= ['LABEL','DEPARTMENT']).count()
#ProductLabels2 = ProductLabels2.drop(ProductLabels2[ProductLabels2['BRAND'] <=10].index)

#palette : 
my_palette = ["maroon","firebrick","crimson", "orangered","chocolate", "peru","sandybrown","rosybrown","darkturquoise"
        ,"c","cadetblue","powderblue","aquamarine","springgreen","limegreen","green","darkkhaki"]
sns.set_palette(my_palette)


fig = plt.figure(figsize=(20,8))

# Divide the figure into a 2x1 grid, and give  the first section
ax1 = fig.add_subplot(121)

# Divide the figure into a 2x1 grid, and give the second section
ax2 = fig.add_subplot(122)

ProductDepartment.BRAND[ProductDepartment['BRAND']>500].unstack().plot.bar(color=sns.color_palette("Paired"),ax= ax1,stacked = True)
ProductLabels2.loc[['DRUG GM','GROCERY','DELI']].BRAND.unstack().plot.bar(color=sns.color_palette(),ax= ax2,stacked = True)


fig.subplots_adjust(hspace=1.3)
fig.subplots_adjust(wspace=0.2)


In the left barplot we observe where we can find the different product types. In x-axis displays the product type and the color label gives the shop (DEPARTMENT) where the product can be found. For example we can see that the households product are mainly found in the drug store (DRUG GM), and most of the meat and seafood products are found in the meat and meat-pckgd shops. 

The right barplot shows the proportions of the different products types, that we can find in the "unspecialized" shops (Deli, the grocery, and the drug store). For example we can see that Deli sales only food products, whereas the drug store proposes an important variety of household products.